In [1]:
# %cd /home/ubu-admin/Developer/tensorflow-wavelets
%cd /workspaces/OpenDVCW
import os
import load
import OpenDVCW
import tensorflow as tf
from math import log10, sqrt
import cv2
import numpy as np

/workspaces/OpenDVCW


In [2]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [3]:
BATCH_SIZE = 1
Height = 240
Width = 240
lmbda = 2048
print("Loading model")
model = tf.keras.models.load_model('model_save_wavelets_L__2048_5_240x240')
# Show the model architecture
model.summary()

Loading model


2022-04-03 16:54:55.631618: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 16:54:55.668469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 16:54:55.669334: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 16:54:55.674860: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "open_dvc"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mv_analysis (Sequential)    (None, None, None, 128)   659456    
                                                                 
 mv_synthesis (Sequential)   (None, None, None, 2)     642824    
                                                                 
 res_analysis (Sequential)   (None, None, None, 128)   1552640   
                                                                 
 res_synthesis (Sequential)  (None, None, None, 3)     1536015   
                                                                 
 wavelets_optical_flow (Wave  multiple                 240050    
 letsOpticalFlow)                                                
                                                                 
 motion_compensation (Motion  multiple                 486467    
 Compensation)                                            

2022-04-03 16:55:50.349965: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [28]:
!rm -r /workspaces/OpenDVCW/Test_com
!mkdir /workspaces/OpenDVCW/Test_com
!mkdir /workspaces/OpenDVCW/Test_com/h264
!mkdir /workspaces/OpenDVCW/Test_com/h265
!mkdir /workspaces/OpenDVCW/Test_com/dvcw

In [29]:

path = load.load_path_n("folder_cloud_test.npy", 0)
p_frame_out_bin = "Test_com/dvcw/p_frame_dvcw.bin"
out_decom = "Test_com/dvcw/frame1.png"
i_on_test = "Test_com/frame0.png"
p_on_test = "Test_com/frame1.png"

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)
print(p_frame)

# write inputs to disk
OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))


OpenDVCW.compress(model, i_frame, p_frame, p_frame_out_bin, 240, 240)
OpenDVCW.decompress(model, i_frame, p_frame_out_bin, out_decom, 240, 240)


original = cv2.imread(p_on_test)
compressed = cv2.imread(out_decom)
bin_size = os.path.getsize(p_frame_out_bin)
value = PSNR(original, compressed)

print("bin size: ", bin_size , "psnr: ", value)

/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00049/0289/im1.png
/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00049/0289/im2.png
compress
decompress
bin size:  2674 psnr:  38.518513072322506


In [32]:
!rm -r /workspaces/OpenDVCW/Test_com/h265/*
!ffmpeg -i /workspaces/OpenDVCW/Test_com/frame%d.png -c:v libx265 -b:v 1k /workspaces/OpenDVCW/Test_com/h265/out.h265
# !ffmpeg -i /workspaces/OpenDVCW/Test_com/h265/out.h265 /workspaces/OpenDVCW/Test_com/h265/h265_decom%d.png


ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

In [31]:
!rm -r /workspaces/OpenDVCW/Test_com/h264/*
!ffmpeg -i /workspaces/OpenDVCW/Test_com/frame%d.png -c:v libx264 -b:v 200k /workspaces/OpenDVCW/Test_com/h264/out.h264
# !ffmpeg -i /workspaces/OpenDVCW/Test_com/h264/out.h264 /workspaces/OpenDVCW/Test_com/h264/h264_decom%d.png

rm: cannot remove '/workspaces/OpenDVCW/Test_com/h264/*': No such file or directory
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --en

In [36]:
!ffprobe -bsfs /workspaces/OpenDVCW/Test_com/h264/out.h264

ffprobe version 4.2.4-1ubuntu0.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-